In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import glob
import re
import json
import os.path
import time
import re


In [2]:
data_set = []

In [3]:
paths = glob.glob("/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/ap89_collection/*")

In [4]:
x = 0
doc_total = 0
for path in paths:
    try:
        fnames = path.split("/")[-1]
        f = open(path,"r", encoding='iso-8859-1')
        text = f.read()
        doc_total = doc_total + text.count('<DOC>')
        for m in zip(re.finditer('<DOC>', text), re.finditer('</DOC>', text)):
            docdata = text[m[0].start():m[1].end()]
            file_data = {}
            for m in zip(re.finditer('<DOCNO>', docdata), re.finditer('</DOCNO>', docdata)):
                doc_no = docdata[m[0].end() + 1 : m[1].start()]
                file_data['DOCNO'] = doc_no

            file_data['TEXT'] = ""
            for m in zip(re.finditer('<TEXT>', docdata), re.finditer('</TEXT>', docdata)):
                file_data['TEXT'] = file_data['TEXT'] + docdata[m[0].end() + 1: m[1].start()]
#             print(file_data)
            data_set.append(file_data)
    except:
        print(path + " error in file generation")

In [52]:
stop_files = open("stoplist.txt", "r")

In [53]:
stopwords = [snowball_stemmer.stem(stopword) for stopword in stop_files.read().split("\n")]

In [5]:
# numbers_checker = re.compile(r"[0-9]+\.[0-9]+|[0-9]+")
# numbers_checker = re.compile(r"([0-9+\.]+$)")
# patern = re.compile(r'?\d+(?:,\d*)?')
# pattern.match("123,12").start()
# "123.134".count(r"[0-9]+.[0-9]+")
# numbers_checker_comma = re.compile(r"([0-9]+,[0-9]+\.[0-9]+|[0-9]+,[0-9]+)")
# numbers_checker = re.compile(r"(\d{1,0})")
# len(re.findall(pattern, "12312"))
# re.findall(numbers_checker, "5110.")

numbers_checker = re.compile(r"(^[\d][\d+\.]+$)")
numbers_checker_comma = re.compile(r"^[\d][\d+\.\,]+$")

In [56]:
vocab = {}
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'[A-Za-z0-9]+')

from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")

docid = 0
docmap = {}
reversedocmap = {}
vocabmap = {}
index = 0
reverse_vocab = {}

file  = open("tokenisation", "w")
start = time.time()
for doc in data_set:
    pos = 0
    docmap[doc['DOCNO']] = docid
    reversedocmap[docid] = doc['DOCNO']
    for token in doc['TEXT'].split():
        token = token.lower()
        if(len(re.findall(numbers_checker, token)) == 1):
            token = token
        elif(len(re.findall(numbers_checker_comma,token)) == 1):
            token = token.replace(",","")
        else:
            # token = snowball_stemmer.stem(re.sub(r"[\.]|[,]|[']|[\"]|[_]|[`]", "", token))
            token = tokenizer.tokenize(token)
            if(len(token) > 0):
                token = token[0]
            else:
                token = ""
            token = snowball_stemmer.stem(token)

        if token not in stopwords :
            if token not in vocab.keys():
                vocab[token] = 0 
                vocabmap[token] = index
                index = index + 1
                reverse_vocab[index] = token

            vocab[token] = vocab[token] + 1
            pos = pos + 1
    #         print("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " , end=', ' )
            file.write("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " )

    docid = docid + 1

end = time.time()
print(end - start)

649.0925583839417


In [62]:
vocab['northwestern']

805

In [60]:
vocabmap['maximum']

3897

In [61]:
reverse_vocab[295]

'northwestern'

In [63]:
cnt = 0
for key in vocab.keys():
    cnt = cnt + vocab[key]

In [64]:
cnt 

20242236

In [66]:
with open('vocab.json', 'w') as outfile:
    json.dump(vocab, outfile)
    
with open('docmap.json', 'w') as outfile:
    json.dump(docmap, outfile)
    
with open('reversedocmap.json', 'w') as outfile:
    json.dump(reversedocmap, outfile)
    
with open('vocabmap.json', 'w') as outfile:
    json.dump(vocabmap, outfile)

    
with open('reverse_vocab.json', 'w') as outfile:
    json.dump(vocabmap, outfile)
